In [1]:
import pandas as pd


df_sentences = pd.read_csv("processed_datasets_hover/sentences_hover.csv")

In [ ]:
import spacy
from fastcoref import spacy_component
# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Add the fastcoref component with LingMessCoref model
nlp.add_pipe(
    "fastcoref", 
    config={
        'model_architecture': 'LingMessCoref', 
        'model_path': 'biu-nlp/lingmess-coref', 
        'device': 'cuda'  # Change to 'cuda' if you have a GPU and the necessary drivers
    }
)

In [3]:
def convert_text_to_sentences(text):
    all_sentences = []
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    all_sentences.append(sentences)
    all_sentences = [item for sublist in all_sentences for item in sublist]
    return all_sentences

In [ ]:
from fastcoref import spacy_component
import spacy

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(
   "fastcoref", 
   config={'model_architecture': 'LingMessCoref', 'model_path': 'biu-nlp/lingmess-coref', 'device': 'cuda'}
)

In [ ]:
text =  'Andy Cohen is the Executive Producer of the Bravo reality television series The Real Housewives, and he also hosts the late-night talk show "Watch What Happens Live with Andy Cohen" on Bravo. '

sentences = convert_text_to_sentences(text)

sentences

In [ ]:
import random

coref_samples = []
raw_samples = []
gpt_samples = []

df_evidence = df_sentences.groupby('gen_evidence')

# Convert the groups to a list
evidence_groups = list(df_evidence)

# Set the random state for reproducibility
rng = random.Random(42)

# Shuffle the groups with the random state
rng.shuffle(evidence_groups)

# Iterate over the shuffled groups
for gen_evidence, group in evidence_groups:
    

    gen_evidence_with_coref = group['gen_evidence_with_coref'].values[0]
    gen_evidence_sentences = convert_text_to_sentences(gen_evidence)
    
    if str(gen_evidence_with_coref).strip() != str(gen_evidence).strip():
        # Only process if the condition is met
        if len(gen_evidence_sentences) > 1:
            print(gen_evidence)
            print(gen_evidence_with_coref)
            
            doc = nlp(      
                gen_evidence, 
                component_cfg={"fastcoref": {'resolve_text': True}}
            )

            resolved_text = doc._.resolved_text
            print(resolved_text)
            
            raw_samples.append(gen_evidence)
            coref_samples.append(resolved_text)
            gpt_samples.append(gen_evidence_with_coref)

            # Stop once we have 300 valid samples
            if len(coref_samples) == 100:
                break


In [14]:
df = pd.DataFrame({
    'Raw Samples': raw_samples,
    'Coref Edit': coref_samples,
    'GPT Edit': gpt_samples,
    'Correct Coref': '',
    'Correct GPT': ''
})

# Export to Excel
df.to_excel('gpt_coref.xlsx', index=False)


In [34]:
df = pd.DataFrame({
    'Raw Sample': raw_samples,
    'Coref Edit': coref_samples,
    'GPT Edit': gpt_samples,
})

df.to_excel('gpt_coref.xlsx', index=False)

In [7]:
import pandas as pd 

df = pd.read_excel('gpt_coref.xlsx')

df = df.loc[(df['Correct Coref'] != 2) & (df['Correct GPT'] != 2)].iloc[:1]

In [9]:
df

,Raw Samples,Coref Edit,GPT Edit,Correct Coref,Correct GPT,Reason
3,The flag of the United States is a prominent ...,The flag of the United States is a prominent ...,The flag of the United States is a prominent e...,1,1,No Change needed


In [8]:
import pandas as pd 

df = pd.read_excel('gpt_coref.xlsx')

#df2 = df.loc[(df['Correct GPT'] == 2)]
#print(len(df2))

#df = df.loc[(df['Correct Coref'] != 2) & (df['Correct GPT'] != 2)]

print(len(df))

df_gpt = df.loc[(df['Correct GPT'] == 1) | (df['Correct GPT'] == 2)]
df_coref = df.loc[(df['Correct Coref'] == 1) ]

print(len(df_gpt))
print(len(df_gpt) / len(df))

print(len(df_coref))
print(len(df_coref) / len(df))


500
488
0.976
103
0.206


In [2]:
import pandas as pd 

df = pd.read_excel('gpt_coref.xlsx')

df_gpt = df.loc[(df['Correct GPT'] == 1) | (df['Correct GPT'] == 2)]

print(len(df_gpt))

df_coref = df.loc[(df['Correct GPT'] != 2) & (df['Correct Coref'] != 2)]
df_gpt_with_coref = df.loc[(df['Correct GPT'] == 1)] 

print(len(df_gpt_with_coref))
print(len(df_coref))

488
115
126


In [ ]:
for gpt_edit, correct in zip(df['GPT Edit'].tolist(), df['Correct GPT'].tolist()):
    print(gpt_edit)
    print(correct)
    print()

In [ ]:
df

In [ ]:
for r_sample, c_sample, gpt_sample in zip(raw_samples, coref_samples, gpt_samples):
    print(r_sample)
    print(c_sample)
    print(gpt_sample)
    print()


In [ ]:
text ="Beautiful is a song by American singer-songwriter Christina Aguilera, featuring rapper Daughtry. It was released in 2002 as the second single from Aguilera's self-titled album Christina Aguilera. The song peaked at number 5 on the US Billboard Hot 100 chart."
 
doc = nlp(      
    text, 
    component_cfg={"fastcoref": {'resolve_text': True}}
)

resolved_text = doc._.resolved_text
resolved_text